In [1]:
# Import findspark and initialise. 
import findspark
findspark.init()

# Extra packages
import urllib.request
import pandas as pd

In [2]:
# 1. Read in the AWS S3 bucket into a DataFrame.

# Download the file
local_file_path = "../data/home_sales_revised.csv"
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
urllib.request.urlretrieve(url, local_file_path)





('../data/home_sales_revised.csv', <http.client.HTTPMessage at 0x223cdc688e0>)

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 2. Create a temporary view of the DataFrame.
# Read the CSV file into a DataFrame
df = spark.read.csv(local_file_path, header=True, inferSchema=True)

# Show the first 5 rows to verify
df.show(5)

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import year, avg, round, format_number

# 1. Filter the DataFrame for four-bedroom houses
four_bedroom_houses = df.filter(df.bedrooms == 4)

# 2. Extract the year from the date column
four_bedroom_houses = four_bedroom_houses.withColumn("year_sold", year(four_bedroom_houses.date))

# 3. Group by the year and calculate the average price
average_price_per_year = four_bedroom_houses.groupBy("year_sold").agg(round(avg("price"), 2).alias("avg_price"))

# 4. Format the avg_price to ensure two decimal places
formatted_result = average_price_per_year.withColumn("avg_price", format_number("avg_price", 2))

# 5. Show the result
print("Average price for four-bedroom houses sold per year:\n")
formatted_result.show()


Average price for four-bedroom houses sold per year:

+---------+----------+
|year_sold| avg_price|
+---------+----------+
|     2022|296,363.88|
|     2019|300,263.70|
|     2020|298,353.78|
|     2021|301,819.44|
+---------+----------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# 1. Filter the DataFrame for homes with 3 bedrooms and 3 bathrooms
three_bedroom_bath_homes = df.filter((df.bedrooms == 3) & (df.bathrooms == 3))

# 2. Group by the year built and calculate the average price
average_price_per_year_built = three_bedroom_bath_homes.groupBy("date_built").agg(round(avg("price"), 2).alias("avg_price"))

# 3. Format the avg_price to ensure two decimal places
formatted_avg_price_per_year_built = average_price_per_year_built.withColumn("avg_price", format_number("avg_price", 2))

# 4. Show the result
print("Average price of homes with 3 bedrooms and 3 bathrooms per year built:\n")
formatted_avg_price_per_year_built.show()


Average price of homes with 3 bedrooms and 3 bathrooms per year built:

+----------+----------+
|date_built| avg_price|
+----------+----------+
|      2015|288,770.30|
|      2013|295,962.27|
|      2014|290,852.27|
|      2012|293,683.19|
|      2016|290,555.07|
|      2010|292,859.62|
|      2011|291,117.47|
|      2017|292,676.79|
+----------+----------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
from pyspark.sql.functions import col

# 1. Filter the DataFrame for homes with 3 bedrooms, 3 bathrooms, two floors, and >= 2,000 sqft
filtered_homes = df.filter(
    (df.bedrooms == 3) &
    (df.bathrooms == 3) &
    (df.floors == 2) &
    (df.sqft_living >= 2000)
)

# 2. Group by the year built and calculate the average price
average_price_per_year_built_filtered = filtered_homes.groupBy("date_built").agg(round(avg("price"), 2).alias("avg_price"))

# 3. Format the avg_price to ensure two decimal places
formatted_avg_price_per_year_built_filtered = average_price_per_year_built_filtered.withColumn("avg_price", format_number("avg_price", 2))

# 4. Show the result
print("Average price of homes with 3 bedrooms, 3 bathrooms, two floors, and >= 2,000 sqft per year built:\n")
formatted_avg_price_per_year_built_filtered.show()



Average price of homes with 3 bedrooms, 3 bathrooms, two floors, and >= 2,000 sqft per year built:

+----------+----------+
|date_built| avg_price|
+----------+----------+
|      2015|297,609.97|
|      2013|303,676.79|
|      2014|298,264.72|
|      2012|307,539.97|
|      2016|293,965.10|
|      2010|285,010.22|
|      2011|276,553.81|
|      2017|280,317.58|
+----------+----------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

from pyspark.sql.functions import avg, round, col
import time

# Start the timer
start_time = time.time()

# 1. Calculate the average price per "view" rating
avg_price_per_view = df.groupBy("view").agg(round(avg("price"), 2).alias("avg_price"))

# 2. Filter to include only those with an average price >= $350,000
filtered_avg_price_per_view = avg_price_per_view.filter(col("avg_price") >= 350000)

# 3. Order by descending view rating
result = filtered_avg_price_per_view.orderBy(col("view").desc())

# 4. Show the result
print("Average price of a home per 'view' rating, where average home price is >= $350,000:\n")
result.show()

# End the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

Average price of a home per 'view' rating, where average home price is >= $350,000:

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.778780460357666 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
# 1. Cache the temporary table
spark.catalog.cacheTable("home_sales")



In [10]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("home_sales")
print(f"Is the home_sales table cached? {is_cached}")

Is the home_sales table cached? True


In [11]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

# Ensure the table is cached
spark.catalog.cacheTable("home_sales")

# Start the timer for the cached query
start_time = time.time()

# Run the query on the cached data
cached_result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")

# Show the result
cached_result.show()

# End the timer and print the runtime for the cached query
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.7215297222137451 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data (switching to csv because of persistant errors with Hadoop dependencies)
pandas_df = df.toPandas()



In [19]:
# 11. Read the formatted parquet data. I have encountered some issues with Hadoop dependencies, so I'm using pandas for this task.
# Read the CSV file into a PySpark DataFrame
output_path = "../data/home_sales_output.csv"  # Specify the path to save the CSV file
pandas_df.to_csv(output_path, index=False)     # Write the DataFrame to a CSV file

print(f"Data has been written to {output_path}.")


Data has been written to ../data/home_sales_output.csv.


In [20]:
# 12. Create a temporary table for the parquet data.
import pandas as pd

# Correct file path to read the CSV file
df_csv = pd.read_csv("../data/home_sales_output.csv")

# Show the first few rows to confirm it was read correctly
print(df_csv.head())

                                     id        date  date_built   price  \
0  f8a53099-ba1c-47d6-9c31-7398aa8f6089  2022-04-08        2016  936923   
1  7530a2d8-1ae3-4517-9f4a-befe060c4353  2021-06-13        2013  379628   
2  43de979c-0bf0-4c9f-85ef-96dc27b258d5  2019-04-12        2014  417866   
3  b672c137-b88c-48bf-9f18-d0a4ac62fb8b  2019-10-16        2016  239895   
4  e0726d4d-d595-4074-8283-4139a54d0d63  2022-01-08        2017  424418   

   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  
0         4          3         3167     11733       2           1    76  
1         2          2         2235     14384       1           0    23  
2         2          2         2127     10575       2           0     0  
3         2          2         1631     11149       2           0     0  
4         3          2         2249     13878       2           0     4  


In [21]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

# Start the timer
start_time = time.time()

# 1. Filter the DataFrame for homes with a price greater than or equal to $350,000
filtered_df = df_csv[df_csv['price'] >= 350000]

# 2. Group by 'view' rating and calculate the average price, rounded to 2 decimal places
result_df = filtered_df.groupby('view').agg(avg_price=('price', 'mean')).round(2)

# 3. Filter the result to include only groups with an average price >= $350,000
final_result = result_df[result_df['avg_price'] >= 350000].sort_values(by='view', ascending=False)

# End the timer
end_time = time.time()

# Show the result
print(final_result)

# Print the runtime
print("--- %s seconds ---" % (end_time - start_time))

       avg_price
view            
100   1026669.50
99    1061201.42
98    1053739.33
97    1129040.15
96    1017815.92
...          ...
4      399631.89
3      398867.60
2      397389.25
1      401044.25
0      403848.51

[101 rows x 1 columns]
--- 0.022623538970947266 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
# Remove the DataFrame from memory
del df_csv

# Force garbage collection
import gc
gc.collect()

print("DataFrame has been removed from memory.")

DataFrame has been removed from memory.


In [23]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? True


In [25]:
# Uncache the table
spark.catalog.uncacheTable("home_sales")

# Verify it's no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False


In [26]:
# Stop the SparkSession
spark.stop()